# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
cw_file= "C:/Users/bbahaneb/Documents/DU Data Bootcamp/Data Bootcamp Homework/data-homework/Python API/Python Api Challenge/WeatherPycityweather.csv"
cw= pd.read_csv(cw_file)
cw_df=pd.DataFrame(cw)
cw_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Humidity,Clouds,Wind Speed (mph)
0,0,ostrovnoy,68.05,39.51,47.64,94,100,4.74
1,1,mahebourg,-20.41,57.70,69.80,77,75,10.29
2,2,qaanaaq,77.48,-69.36,21.06,78,36,9.86
3,3,zhuhai,22.28,113.57,84.81,83,40,6.93
4,4,cape town,-33.93,18.42,53.78,87,75,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(api_key=g_key)

In [31]:
fig=gmaps.figure()

heatmap_layer=gmaps.heatmap_layer(
    cw_df[["Latitude", "Longitude"]], weights=cw_df["Humidity"], max_intensity=100, point_radius=5, dissipating=False)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
best_temp= cw_df.loc[(cw_df["Temperature (F)"]>=30) & (cw_df["Temperature (F)"]<=72)]
best_temp

best_humid= best_temp.loc[best_temp["Humidity"]<60]
best_humid

best_weather= best_humid.loc[(best_humid["Clouds"]<50) & (best_humid["Clouds"]>10)]
best_weather

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Humidity,Clouds,Wind Speed (mph)
6,6,flinders,-34.58,150.86,68.72,55,40,8.05
141,141,sanchazi,42.08,126.60,65.32,59,42,0.69
315,315,miraflores,19.22,-98.81,69.10,43,40,14.99
365,365,ulladulla,-35.35,150.47,65.03,55,40,8.05
388,388,geraldton,-28.77,114.60,69.80,28,25,21.92
396,396,nelson bay,-32.72,152.15,71.11,49,20,13.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df=pd.DataFrame()
hotel_df["Hotel Name"]=""
    
hotel_df["Latitude"]=best_weather["Latitude"]
hotel_df["Longitude"]=best_weather["Longitude"]
hotel_df["City"]=best_weather["City"]
hotel_df.reset_index(drop=True, inplace=True)
country=["Australia", "China", "Mexico", "Australia", "Australia", "Australia"]
hotel_df["Country"]=country

hotel_df

,Hotel Name,Latitude,Longitude,City,Country
0,NaN,-34.58,150.86,flinders,Australia
1,NaN,42.08,126.60,sanchazi,China
2,NaN,19.22,-98.81,miraflores,Mexico
3,NaN,-35.35,150.47,ulladulla,Australia
4,NaN,-28.77,114.60,geraldton,Australia
5,NaN,-32.72,152.15,nelson bay,Australia


In [47]:
lats=[]
lngs=[]

for each in range(len(hotel_df["City"])):
    lats.append(hotel_df["Latitude"][each])
    lngs.append(hotel_df["Longitude"][each])


hotels=[]

for each in range(len(hotel_df["City"])):
    url= f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?type=establishment&keyword=hotel&fields=name&key={g_key}&radius=5000&location="
    query_url=url+str(lats[each])+","+str(lngs[each])
    response=requests.get(query_url).json()
    hotels.append(response["results"][0]["name"])
    

hotel_df["Hotel Name"]=hotels

hotel_df.to_csv(r"C:/Users/bbahaneb/Documents/DU Data Bootcamp/Data Bootcamp Homework/data-homework/Python API/Python Api Challenge/WeatherPy/hotels.csv")
hotel_df

,Hotel Name,Latitude,Longitude,City,Country
0,Dream Catcher Beach House,-34.58,150.86,flinders,Australia
1,Sengong Hotel,42.08,126.60,sanchazi,China
2,Hotel Mansión,19.22,-98.81,miraflores,Mexico
3,Mollymook Surfbeach Motel & Apartments,-35.35,150.47,ulladulla,Australia
4,The Gerald Apartment Hotel,-28.77,114.60,geraldton,Australia
5,Amarna Luxury Beach Resort - Guest Only,-32.72,152.15,nelson bay,Australia


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [50]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure

Figure(layout=FigureLayout(height='420px'))